In [1]:
import numpy as np

In [6]:
def linear(X, W):
    H = np.dot(X, W)
    
    return H

In [7]:
def sigmoid(H):
    p = 1 / (1 + np.exp(-H))
    
    return p

In [8]:
X = np.array([1, 2])

In [9]:
W = np.array([2, 3])

In [13]:
H = linear(X, W)
print(H)
p = sigmoid(H)
print(p)

8
0.99966464987


In [14]:
W = np.array([-4, -3])

In [15]:
H = linear(X, W)
print(H)
p = sigmoid(H)
print(p)

-10
4.53978687024e-05


In [16]:
from sklearn.datasets import load_iris

In [17]:
data = load_iris()

In [18]:
x = data["data"]
y = data["target"]

In [41]:
x[:10]

array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2],
       [ 5.4,  3.9,  1.7,  0.4],
       [ 4.6,  3.4,  1.4,  0.3],
       [ 5. ,  3.4,  1.5,  0.2],
       [ 4.4,  2.9,  1.4,  0.2],
       [ 4.9,  3.1,  1.5,  0.1]])

In [44]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [45]:
class Softmax:
    def __init__(self):
        
        self.params = {}
        self.params['W'] = 0.0001 * np.random.randn(4, 3)  #4x3 행렬로 가중치 w 랜덤 생성(초기 가중치 설정)
        self.params['b'] = np.ones(3)  #1x3 배열에 모두 1 넣기 #역시 linear function의 상수값에 대해 초기값 지정 
    
    def forward(self, X):
        W = self.params['W']  #초기값 지정 
        b = self.params['b']

        h = np.dot(X, W) + b  #linear function 
        a = np.exp(h)  #softmax로 확률 구하기 위해 exponential 취하기 
        #stable_a = np.exp(h - np.max(h, axis = 1).reshape(-1,1))  #loss nan값 방지 
        p = a/np.sum(a, axis = 1).reshape(-1,1)  #softmax로 해당 label이 될 확률 p구하기   #1열에 맞게 행은 가변적으로 지정
        return p
    
    def loss(self, X, T):  #좋은 가중치 w를 구하기 위해 loss구하는 function 구현!
        
        p = self.forward(X)  #기존 가중치를 이용해 확률p 구하기 
        
        n = T.shape[0]  #실제 label의 행 수 저장 
        log_likelihood = 0
        log_likelihood -= np.log(p[np.arange(n), T]).sum()  #현재 구한 p를 softmax - Cross_entropy에 대해 미분 #equals P-T
        Loss = log_likelihood / n

        return Loss
    
    def accuracy(self, X, T):
        p = self.forward(X) #예측
        predict = np.argmax(p, axis = 1) #예측 결과 #확률이 가장 높은 인덱스가 예측된 label!
        
        return 1 - np.count_nonzero(predict - T)/len(T)
        
    def gradient(self, X, T, learning_rate = 0.0001):
        
        p = self.forward(X)
        
        t = np.zeros((T.shape[0], 3))
        t[np.arange(T.shape[0]), T] = 1
        #t는 인덱스 레이블 T를 One hot 벡터로 바꾼 것
        
        dp = p.copy()
        dp[np.arange(len(T)), T] -= 1 #P 미분
        
        #목적함수에 대한 가중치 미분값을 담을 zero array 생성
        grads = {}
        grads['W'] = np.zeros((4, 3))
        grads['b'] = np.zeros(10)
        #목적함수에 대한 가중치 미분값 합 구하기
        grads['W'] = (1/len(T)) * np.dot(X.T, p-t)  #chain rule 적용해 Loss의 w(가중치)에 대한 미분값 
        grads['b'] = (1/len(T)) * np.sum(p-t, axis = 0)
        #p-t 대신 dp 사용 가능
        
        self.params['W'] -= learning_rate * grads['W']  #loss를 minimize하는 방향으로 업데이트 
        self.params['b'] -= learning_rate * grads['b']

In [46]:
softmax = Softmax()

In [47]:
for i in range(5000):
    softmax.gradient(x, y)
    
    if i % 1000 == 0:
        print(i, "번째 학습중입니다.")
        print("Accuracy : ", softmax.accuracy(x, y))
        print("Loss :     ", softmax.loss(x, y))
        print(softmax.params['W'])

0 번째 학습중입니다.
Accuracy :  0.6466666666666667
Loss :      1.09855297238
[[  1.96573693e-04   4.93914075e-05   6.76184218e-05]
 [  1.30445304e-04   2.14928839e-04   2.37028269e-05]
 [ -7.80052698e-05   1.18282549e-04   3.06657993e-05]
 [  8.95771398e-05  -2.16292619e-06   1.15314668e-04]]
1000 번째 학습중입니다.
Accuracy :  0.33999999999999997
Loss :      1.02009961798
[[-0.00302049  0.0006832   0.00265087]
 [ 0.02415693 -0.01042444 -0.01336342]
 [-0.05790276  0.01478975  0.04318395]
 [-0.02533815  0.00350206  0.02203882]]
2000 번째 학습중입니다.
Accuracy :  0.6666666666666667
Loss :      0.964618405164
[[ 0.00879421 -0.00032785 -0.00815277]
 [ 0.05422972 -0.02155237 -0.03230827]
 [-0.10198737  0.02761763  0.07444068]
 [-0.04584327  0.00632463  0.03972137]]
3000 번째 학습중입니다.
Accuracy :  0.6666666666666667
Loss :      0.916372262449
[[ 0.0217381  -0.00156037 -0.01986415]
 [ 0.0833534  -0.03239023 -0.05059409]
 [-0.14192619  0.03945218  0.10254495]
 [-0.06458018  0.00873061  0.0560523 ]]
4000 번째 학습중입니다.
Accu